# Prueba Word

In [37]:
import pandas as pd
from docxtpl import DocxTemplate, InlineImage
from docx import Document
from docx.enum.text import WD_BREAK
from io import BytesIO
from docx.shared import Mm

In [ ]:
# Load Excel
df = pd.read_excel("datos_estantería.xlsx")

# Create a blank master document that will hold all pages
master = Document("plantilla_master.docx")

for i, row in df.iterrows():
    # Load the template for this row
    tpl = DocxTemplate("plantilla_estanteria.docx")

    # Fill in placeholders with row data
    context = {
        'id': row['id'],
        'descripcion': row['descripcion'],
        'aviso_estatica': pd.isna(row['nivel']),
        'nivel': str(row['nivel'])    }

    tpl.render(context)

    # Save the rendered template to memory (not disk)
    temp_stream = BytesIO()
    tpl.save(temp_stream)
    temp_stream.seek(0)

    # Open the rendered docx and copy its content into the master document
    temp_doc = Document(temp_stream)

    for element in temp_doc.element.body:
        master.element.body.append(element)

    # Add page break (except after the last one)
    if i < len(df) - 1:
        master.add_page_break()

# Save the final combined Word file
master.save("combined_output.docx")


In [26]:
df['aviso_estatica'] = ~pd.isna(df.nivel)
df.head(2)

,id,descripcion,nivel,aviso_estatica
0,ES45,Una estantería muy chula,AMARILLO,True
1,ES46,Una estantería muy rota,NaN,False


In [29]:
df.nivel.unique()

array(['AMARILLO', nan, 'VERDE', 'ROJO'], dtype=object)

In [30]:
colors = {"AMARILLO": "#F5C242", "VERDE": "#4EAD5B", "ROJO": "#EA3323"}

In [33]:
df["bg"] = df.nivel.apply(lambda x: colors[x] if not pd.isna(x) else None)

In [ ]:
context['estanterias'][0]['image']

In [50]:
import jinja2
tpl = DocxTemplate("plantilla_master.docx")
df["image"] = [InlineImage(tpl, image_descriptor="image_2.png", width=Mm(70))] * df.shape[0]
context = {"estanterias": list(df.T.to_dict().values())}
jinja_env = jinja2.Environment(autoescape=True)
tpl.render(context)
tpl.save("combined_output.docx")